In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train_identity.csv
/kaggle/input/train_transaction.csv
/kaggle/input/test_transaction.csv
/kaggle/input/test_identity.csv
/kaggle/input/sample_submission.csv


# Read train(test) transaction & identity

In [11]:
train_transaction= pd.read_csv('../input/train_transaction.csv')
train_identity = pd.read_csv('../input/train_identity.csv')

test_transaction = pd.read_csv('../input/test_transaction.csv')
test_identity = pd.read_csv('../input/test_identity.csv')

print('train_transaction.shape: {}\ntest_transaction.shape: {}'.format(train_transaction.shape, test_transaction.shape))

print('\ntrain_identity.shape: {}\ntest_identity.shape: {}'.format(train_identity.shape, test_identity.shape))


train_transaction.shape: (590540, 394)
test_transaction.shape: (506691, 393)

train_identity.shape: (144233, 41)
test_identity.shape: (141907, 41)


In [5]:
print('train_transaction[TransactionID] 데이터 개수: ', \
      train_transaction['TransactionID'].value_counts().sum())

print('\ntrain_identity[TransactionID] 데이터 개수: ', \
      train_identity['TransactionID'].value_counts().sum())

print('\ntest_transaction[TransactionID] 데이터 개수: ', \
      test_transaction['TransactionID'].value_counts().sum())

print('\ntest_identity[TransactionID] 데이터 개수: ', \
      test_identity['TransactionID'].value_counts().sum())


train_transaction[TransactionID] 데이터 개수:  590540

train_identity[TransactionID] 데이터 개수:  144233

test_transaction[TransactionID] 데이터 개수:  506691

test_identity[TransactionID] 데이터 개수:  141907


In [10]:
# 셀 실행결과를 보면.. train_identity의 데이터 로우 개수와 같다.
print('[train 데이터셋] 중복 TransactionID 개수: ',\
     np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'].unique())))

# 셀 실행결과를 보면.. test_identity의 데이터 로우 개수와 같다.
print('[test 데이터셋] 중복 TransactionID 개수: ',\
     np.sum(test_transaction['TransactionID'].isin(test_identity['TransactionID'].unique())))

[train 데이터셋] 중복 TransactionID 개수:  144233
[test 데이터셋] 중복 TransactionID 개수:  141907


In [17]:
# transaction, identity 파일이 갖는 공통속성은 TransactionID이므로
# TransactionID 로 두 파일을 합친다. (outer join)
train_merged= pd.merge(train_transaction, 
                       train_identity, 
                       on='TransactionID', 
                       how='outer')

# 나중을 위하여 test 데이터셋도 합친다.
test_merged = pd.merge(test_transaction, 
                       test_identity, 
                       on='TransactionID',
                       how='outer')

train_merged.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.0,H,2803,100.0,150.0,visa,226.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009,0,86536,117.0,W,17399,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
train_columns= train_merged.columns


Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)

# Modeling

RandomForest, DecisionTree 분류기로 학습..

In [ ]:
# # 아직 돌리면 안됨 돌리면 죽어 ^^
# from sklearn.model_selection import train_test_split

# #학습데이터셋 정답레이블
# y_train_df=train_merged['isFraud'] 

# #학습데이터셋(훈련데이터셋): TransactionID, isFraud 컬럼을 제외한 나머지 컬럼들을 수집한다.
# X_train_df=train_merged.drop(['TransactionID','isFraud'], axis=1)

# # 실제 테스트 데이터셋
# # TransactionID를 제외한 나머지만을 ...
# test_transactionID = test_merged['TransactionID'] #결과파일에 붙이기 위하여..
# X_test_df= test_merged.drop('TransactionID', axis=1)

In [ ]:
# # 돌리면 안되, 죽어 ^^
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier

# #결정트리, RandomForest 사이킷런 Classifier 클래스 생성
# dt_clf = DecisionTreeClassifier(random_state=11)
# rf_clf = RandomForestClassifier(random_state=11)

# # 1. 결정트리
# # 결정트리 학습/예측/평가
# dt_clf.fit(X_train_df, y_train_df) #학습
# dt_isFraud=dt_clf.predict(X_test_df) #예측

# # 결과 DataFrame을 생성한다.
# dt_result={
#     'TransactionID': test_transactionID,
#     'isFraud': dt_isFraud
#           }
# # csv파일을 생성하여 올린다.
# dt_result.to_csv('dt_submission.csv', index=False)



# # 2. RandomForest
# rf_clf.fit(X_train_df, y_train_df) #학습
# rf_isFraud=rf_clf.predict(X_test_df) #예측

# # 결과 DataFrame을 생성한다.
# rf_result={
#     'TransactionID': test_transactionID,
#     'isFraud': rf_isFraud
#           }
# # csv파일을 생성하여 올린다.
# rf_result.to_csv('rf_submission.csv', index=False)
